In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib
import scanpy.api as sc
import scipy
def readData_SingleR(path,min_genes):
    """ Returns single cell data as a matrix. 
    
    Parameters
    ----------
    path : str
        Diractory path, the location of single cell data.
    min_genes : int
        The minimum number of genes for a cell to have in order to participate the analysis.
    Returns
    -------
    sc_data : DataFrame
        Single cell data matrix. 
    """
    data = sc.read(path + 'matrix.mtx').transpose() #, cache=True
    data.var_names = np.genfromtxt(path + 'genes.tsv', dtype=str)[:, 1]
    data.obs_names = np.genfromtxt(path + 'barcodes.tsv', dtype=str)
    data.var_names_make_unique()
    data.obs['n_counts'] = np.sum(data.X, axis=1).A1
    sc.pp.filter_cells(data, min_genes=min_genes)

    sc_data=pd.DataFrame(data.X.toarray())
    sc_data.index=data.obs_names.values
    sc_data.columns=data.var_names.values
    sc_data=sc_data.T
    sc_data
    return sc_data

def getDEgenes(refDataset,anot=None,n=None):
    """ Create a list for differential genes for all pairwise cell types in the a reference data set.
    
    Parameters
    ----------
    refDataset : DataFrame
        The reference dataset gene expression matrix.
    anot : DataFrame
        Annotations for each column in ref_data.
    n : int
        Number of top differential genes.
    Returns
    -------
    deGenes : dict with multiple index
        Dictionary containing differential genes for each combination of cell types. 
    """
    if n is None:
        n=int(500*np.power(2/3,np.log2(len(np.unique(anot.cellType)))))
    if anot is not None:
        types=np.unique(anot.cellType)
        median=refDataset.groupby(anot.cellType.values,axis=1).apply(np.median,axis=1)
        deGenes={}
        [deGenes.update({(i,j):median[i]-median[j]}) for i in median.index  for j in median.index if i!=j]
        for i in deGenes.keys():
            deGenes[i]=refDataset.iloc[deGenes.get(i).argsort()[-n:]].index.values.tolist()
    else:
        anot=refDataset.loc["cell_type"].values
        types=np.unique(anot)
        refDataset=refDataset.drop("cell_type")
        refDataset=refDataset.astype(float)
        median=refDataset.groupby(anot,axis=1).apply(np.median,axis=1)
        deGenes={}
        [deGenes.update({(i,j):median[i]-median[j]}) for i in median.index  for j in median.index if i!=j]
        for i in deGenes.keys():
            deGenes[i]=refDataset.iloc[deGenes.get(i).argsort()[-n:]].index.values.tolist()
    return deGenes

def getClassificationScores(sc_data,refDataset,anot):
    """ Returns a matrix of scores of spearman correlation with reference classes for each single cell. 
    
    Parameters
    ----------
    sc_data : DataFrame
        Sc-RNAseq data.
    refDataset : DataFrame
        The reference dataset gene expression matrix.
    anot : DataFrame
        Annotations for each column in ref_data.
    Returns
    -------
    scores : tuple
        A tuple object including scoring matrix and correlation matrix. 
    """
    intersect=np.intersect1d(refDataset.index.values,sc_data.index)
    sc_data=sc_data.loc[intersect]
    refDataset=refDataset.loc[intersect]
    output=pd.DataFrame(index=np.unique(anot["cellType"]))
    
    de=getDEgenes(refDataset,anot=anot)
    de_merged=[]
    [de_merged.extend(i) for i in  de.values()]
    de_merged=np.unique(de_merged) #the results before here is exactly the same but here the length differs IDK why.

    cor=scipy.stats.spearmanr(sc_data.loc[de_merged],refDataset.loc[de_merged])
    cor=pd.DataFrame(cor[0]).iloc[:,0:len(sc_data.columns)][-len(refDataset.columns):]
    cor.columns=sc_data.columns
    cor.index=refDataset.columns
    cor["cellType"]=anot["cellType"].values
    scores=cor.groupby("cellType").quantile(q=0.8)
    return scores,cor

In [2]:
Sample_sc=readData_SingleR(os.getcwd()+"\\Datasets\\b_cells_filtered_gene_bc_matrice\\",500)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [3]:
refDataset=pd.DataFrame.from_csv(os.getcwd()+"\\ref_sample_filtered.csv")
anot=pd.DataFrame.from_csv(os.getcwd()+"\\anot_sample.csv")
anot.columns=["cellType"]

C:\Users\murat_gga8ya6\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.
C:\Users\murat_gga8ya6\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  


In [4]:
scores=getClassificationScores(Sample_sc,refDataset,anot)

In [6]:
scores[0].idxmax()

0.8
AAACATACAATGCC-1           CD8 T Cell
AAACATACACGCAT-1           CD8 T Cell
AAACATACGAATAG-1        Memory T cell
AAACATACGTGTCA-1         B Cell naive
AAACATTGGCCCTT-1         B Cell naive
AAACATTGTTTGGG-1         B Cell naive
AAACCGTGGGGACA-1        Memory T cell
AAACCGTGTGCATG-1         B Cell naive
AAACGCACCCAGTA-1           CD8 T Cell
AAACGCACGCCCTT-1        Memory T cell
AAACGCACGTACCA-1        Memory T cell
AAACGCACGTGTAC-1           CD8 T Cell
AAACGCTGATCTTC-1         B Cell naive
AAACGCTGCCCTCA-1           CD8 T Cell
AAACGCTGCCTACC-1           CD8 T Cell
AAACGCTGGCCTTC-1        Memory T cell
AAACGGCTTGTCAG-1           CD8 T Cell
AAACTTGAACGCTA-1         B Cell naive
AAACTTGAACGTAC-1         B Cell naive
AAACTTGATCTTAC-1        Memory T cell
AAACTTGATGCCCT-1         B Cell naive
AAAGACGACAATCG-1              NK Cell
AAAGACGACAGAGG-1         B Cell naive
AAAGACGACCAATG-1           CD8 T Cell
AAAGACGACCTTAT-1         B Cell naive
AAAGAGACCGTCTC-1         B Cell naive
AAAGAGAC